In [ ]:
connection_URI="bolt://localhost:11003"
username="neo4j"
password="131kalana"

In [ ]:
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response


In [ ]:
def check_process_exists(split_string):
    
    for curr in processes_list:
        if(curr['PID']) == split_string[1]:
            temp = (" ".join(split_string[4:len(split_string)])).split(";")
            curr['Executable'].append("\n" + "\n".join(temp))
            return True
    
    temp = (" ".join(split_string[4:len(split_string)])).split(";")
    processes_list.append({"PID": split_string[1], "User": split_string[2],
                "Group": split_string[3], "Executable": ["\n" + "\n".join(temp)], "Children": []})
    return False

In [ ]:
def add_child_process(PPID, PID):
    for curr in processes_list:
        if curr['PID'] == PPID:
            if PID not in curr['Children']:
                curr['Children'].append(PID)

In [ ]:
def read_and_convert_to_JSON():
    # this flag will help to skip parent id of the first process
    is_first_line_read = False 
    with open("processtrace.txt") as trace_file:
        for line in trace_file:
            if is_first_line_read == False:
                is_first_line_read = True
                split_string = line.split()
                # manually adding the bash parent
                temp = (" ".join(split_string[4:len(split_string)])).split(";")
                processes_list.append({"PID": split_string[1], "User": split_string[2],
                "Group": split_string[3], "Executable": ["\n"+"\n".join(temp) ], "Children": []})
            else:
                split_string = line.split()
                check_process_exists(split_string)

    is_first_line_read = False                
    with open("processtrace.txt") as trace_file:
        for line in trace_file:
            if is_first_line_read == False:
                is_first_line_read = True
            else:
                split_string = line.split()
                add_child_process(split_string[0], split_string[1])
                
    

In [ ]:
def main():
    read_and_convert_to_JSON()
    conn = Neo4jConnection(uri=connection_URI, user=username, pwd=password)
    conn.query("CREATE OR REPLACE DATABASE processTrace")
    #clear all the data in the database
    conn.query("match (n) detach delete (n)")
    # this flag helps to find the first element, I.e parent point/ entry process
    is_first = True
    # this loop will add nodes to the graphing database
    for curr_process in processes_list:
        execs = '\n'.join(curr_process['Executable'])
        if is_first:
            conn.query("create(n:entry_process{_pid: " + curr_process["PID"] + ", User: '" + curr_process["User"] +
            "', Group: '" + curr_process["Group"] + "', Executable: '" + execs + "'})")
            is_first = False
        else:
            conn.query("create(n:process{_pid: " + curr_process["PID"] + ", User: '" + curr_process["User"] +
            "', Group: '" + curr_process["Group"] + "', Executable: '" + execs + "'})")
    # now lets create relationships
    is_first = True
    for curr in processes_list:
        if is_first: # becasue entry process is always at the begining of the list
            is_first = False
            for curr_child in curr['Children']:
                conn.query("match (n:entry_process), (p:process) where n._pid="+curr['PID'] +
                 " and p._pid=" + curr_child + " create (n)-[r:spawns]->(p)")
        else:
           for curr_child in curr['Children']:
                conn.query("match (n:process), (p:process) where n._pid="+curr['PID'] +
                 " and p._pid=" + curr_child + " create (n)-[r:spawns]->(p)") 
                

In [ ]:
if __name__ == "__main__":
    main()



In [ ]:
import webbrowser
webbrowser.open("view-from-the-adversarys-perspective.html")